In [0]:
from pyspark.sql.functions import * 
from pyspark.sql.types import *

## Silver Layer Script

### Data Access Using App

In [0]:
spark.conf.set("fs.azure.account.auth.type", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")

# Client ID and Secret retrieved from Databricks secrets
spark.conf.set("fs.azure.account.oauth2.client.id", dbutils.secrets.get("adls-scope", "client-id"))
spark.conf.set("fs.azure.account.oauth2.client.secret", dbutils.secrets.get("adls-scope", "client-secret"))
spark.conf.set("fs.azure.account.oauth2.client.endpoint", "https://login.microsoftonline.com/<tenant-id>/oauth2/token")


### Data Loading

#####Reading Data

In [0]:
df_cal = spark.read.format('csv')\
    .option("header",True)\
    .option("inferschema",True)\
    .load('abfss://bronze@awstoragedatalake2026.dfs.core.windows.net/AdventureWorks_Calendar')

In [0]:
df_cus = spark.read.format('csv')\
            .option("header",True)\
            .option("inferSchema",True)\
            .load('abfss://bronze@awstoragedatalake2026.dfs.core.windows.net/AdventureWorks_Customers')

In [0]:
df_procat = spark.read.format('csv')\
            .option("header",True)\
            .option("inferSchema",True)\
            .load('abfss://bronze@awstoragedatalake2026.dfs.core.windows.net/AdventureWorks_Product_Categories')

In [0]:
df_pro = spark.read.format('csv')\
            .option("header",True)\
            .option("inferSchema",True)\
            .load('abfss://bronze@awstoragedatalake2026.dfs.core.windows.net/AdventureWorks_Products')

In [0]:
df_ret = spark.read.format('csv')\
            .option("header",True)\
            .option("inferSchema",True)\
            .load('abfss://bronze@awstoragedatalake2026.dfs.core.windows.net/AdventureWorks_Returns')

In [0]:
df_sales = spark.read.format('csv')\
            .option("header",True)\
            .option("inferSchema",True)\
            .load('abfss://bronze@awstoragedatalake2026.dfs.core.windows.net/AdventureWorks_Sales*')

In [0]:
df_ter = spark.read.format('csv')\
            .option("header",True)\
            .option("inferSchema",True)\
            .load('abfss://bronze@awstoragedatalake2026.dfs.core.windows.net/AdventureWorks_Territories')

In [0]:
df_subcat = spark.read.format('csv')\
            .option("header",True)\
            .option("inferSchema",True)\
            .load('abfss://bronze@awstoragedatalake2026.dfs.core.windows.net/Product_Subcategories')

###Transformations

#### Calendar

In [0]:
df_cal.display()

In [0]:
df_cal = df.withColumn('Month',month(col('Date')))\
            .withColumn('Year',year(col('Date')))

In [0]:
df_cal.display()

In [0]:
df_cal.write.format('parquet')\
            .mode('append')\
            .option("path","abfss://silver@awstoragedatalake2026.dfs.core.windows.net/AdventureWorks_Calendar")\
            .save()

####Customers

In [0]:
df_cus.display()

In [0]:
df_cus.withColumn("fullName",concat(col('Prefix'),lit(' '),col('FirstName'),lit(' '),col('LastName'))).display()

In [0]:
df_cus = df_cus.withColumn('fullName',concat_ws(' ',col('Prefix'),col('FirstName'),col('lastName')))

In [0]:
df_cus.write.format('parquet')\
            .mode('append')\
            .option("path","abfss://silver@awstoragedatalake2026.dfs.core.windows.net/AdventureWorks_Customers")\
            .save()

#### Sub Categories

In [0]:
df_subcat.display()

In [0]:
df_subcat.write.format('parquet')\
            .mode('append')\
            .option("path","abfss://silver@awstoragedatalake2026.dfs.core.windows.net/AdventureWorks_SUbCategories")\
            .save()

####Products

In [0]:
df_pro.display()

In [0]:
df_pro = df_pro.withColumn('ProductSKU',split(col('ProductSKU'),'-')[0])\
                .withColumn('ProductName',split(col('ProductName'),' ')[0])

In [0]:
df_pro.display()

In [0]:
df_pro.write.format('parquet')\
            .mode('append')\
            .option("path","abfss://silver@awstoragedatalake2026.dfs.core.windows.net/AdventureWorks_Products")\
            .save()

####Returns

In [0]:
df_ret.display()

In [0]:
df_ret.write.format('parquet')\
            .mode('append')\
            .option("path","abfss://silver@awstoragedatalake2026.dfs.core.windows.net/AdventureWorks_Returns")\
            .save()

####Territories

In [0]:
df_ter.display()

In [0]:
df_ter.write.format('parquet')\
            .mode('append')\
            .option("path","abfss://silver@awstoragedatalake2026.dfs.core.windows.net/AdventureWorks_Territories")\
            .save()

####Sales

In [0]:
df_sales.display()

In [0]:
df_sales = df_sales.withColumn('StockDate',to_timestamp('StockDate'))

In [0]:
df_sales = df_sales.withColumn('OrderNumber',regexp_replace(col('OrderNumber'),'S','T'))

In [0]:
df_sales = df_sales.withColumn('multiply',col('OrderLineItem')*col('OrderQuantity'))

In [0]:
df_sales.display()

In [0]:
df_sales.write.format('parquet')\
            .mode('append')\
            .option("path","abfss://silver@awstoragedatalake2026.dfs.core.windows.net/AdventureWorks_Sales")\
            .save()

####Sales Analysis

In [0]:
df_sales.groupBy('OrderDate').agg(count('OrderNumber').alias('total_order')).display()

Databricks visualization. Run in Databricks to view.

In [0]:
df_procat.display()

Databricks visualization. Run in Databricks to view.

In [0]:
df_ter.display()

Databricks visualization. Run in Databricks to view.